In [ ]:
!gdown --id 10j65AwoGzzd2lp8bReWoDxNc_OgBX4nG

Downloading...
From: https://drive.google.com/uc?id=10j65AwoGzzd2lp8bReWoDxNc_OgBX4nG
To: /content/Finalmodel.zip
55.9MB [00:00, 64.5MB/s]


In [ ]:
!unzip '/content/Finalmodel.zip'

Archive:  /content/Finalmodel.zip
   creating: Finalmodel/image-captioning-keras-resnet/
   creating: Finalmodel/image-captioning-keras-resnet/.git/
   creating: Finalmodel/image-captioning-keras-resnet/.git/branches/
  inflating: Finalmodel/image-captioning-keras-resnet/.git/config  
  inflating: Finalmodel/image-captioning-keras-resnet/.git/description  
 extracting: Finalmodel/image-captioning-keras-resnet/.git/HEAD  
   creating: Finalmodel/image-captioning-keras-resnet/.git/hooks/
  inflating: Finalmodel/image-captioning-keras-resnet/.git/hooks/applypatch-msg.sample  
  inflating: Finalmodel/image-captioning-keras-resnet/.git/hooks/commit-msg.sample  
  inflating: Finalmodel/image-captioning-keras-resnet/.git/hooks/fsmonitor-watchman.sample  
  inflating: Finalmodel/image-captioning-keras-resnet/.git/hooks/post-update.sample  
  inflating: Finalmodel/image-captioning-keras-resnet/.git/hooks/pre-applypatch.sample  
  inflating: Finalmodel/image-captioning-keras-resnet/.git/hooks/pr

In [ ]:
!cd /content/image-captioning-keras-resnet

/bin/bash: line 0: cd: /content/image-captioning-keras-resnet: No such file or directory


In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install PyDictionary

  Created wheel for goslate: filename=goslate-1.5.1-py3-none-any.whl size=11411 sha256=d6a457b45d2b8dfd91ab1c93addd79c0e963216446ed6f84bc48daafdc36acad
  Stored in directory: /root/.cache/pip/wheels/16/5c/86/f561bb944dc9f853f9502427bde8fe557a6fafe146d4a432f2
Successfully built goslate


In [17]:
from flask import Flask, render_template, request
import tensorflow as tf
import cv2
from keras.models import load_model
import numpy as np
from keras.applications.resnet import ResNet50
from keras.optimizers import Adam
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from keras.utils import np_utils
from keras.preprocessing import image, sequence
import cv2
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from flask_ngrok import run_with_ngrok
import nltk
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
import re


vocab = np.load('/content/Finalmodel/image-captioning-keras-resnet/vocab.npy', allow_pickle=True)

vocab = vocab.item()

inv_vocab = {v:k for k,v in vocab.items()}


print("+"*50)
print("vocabulary loaded")


embedding_size = 128
vocab_size = len(vocab)
max_len = 40


image_model = Sequential()

image_model.add(Dense(embedding_size, input_shape=(2048,), activation='relu'))
image_model.add(RepeatVector(max_len))


language_model = Sequential()

language_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))


conca = Concatenate()([image_model.output, language_model.output])
x = LSTM(128, return_sequences=True)(conca)
x = LSTM(512, return_sequences=False)(x)
x = Dense(vocab_size)(x)
out = Activation('softmax')(x)
model = Model(inputs=[image_model.input, language_model.input], outputs = out)

model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

model.load_weights('/content/Finalmodel/image-captioning-keras-resnet/mine_model_weights.h5')

print("="*150)
print("MODEL LOADED")

resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')


#resnet = load_model('/content/image-captioning-keras-resnet/model.h5')

print("="*150)
print("RESNET MODEL LOADED")




app = Flask(__name__,template_folder="/content/Finalmodel/image-captioning-keras-resnet/templates",static_url_path='/content/Finalmodel/image-captioning-keras-resnet/static')
run_with_ngrok(app)

app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 1


@app.route('/')
def index():
    return render_template('index.html')

@app.route('/after', methods=['GET', 'POST'])
def after():

    global model, resnet, vocab, inv_vocab

    img = request.files['file1']

    img.save('/content/Finalmodel/image-captioning-keras-resnet/static/file.jpg')

    print("="*50)
    print("IMAGE SAVED")


    
    image = cv2.imread('/content/Finalmodel/image-captioning-keras-resnet/static/file.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = cv2.resize(image, (224,224))

    image = np.reshape(image, (1,224,224,3))

    
    
    incept = resnet.predict(image).reshape(1,2048)

    print("="*50)
    print("Predict Features")


    text_in = ['startofseq']

    final = ''

    print("="*50)
    print("GETING Captions")

    count = 0
    while tqdm(count < 20):

        count += 1

        encoded = []
        for i in text_in:
            encoded.append(vocab[i])

        padded = pad_sequences([encoded], maxlen=max_len, padding='post', truncating='post').reshape(1,max_len)

        sampled_index = np.argmax(model.predict([incept, padded]))

        sampled_word = inv_vocab[sampled_index]

        if sampled_word != 'endofseq':
            final = final + ' ' + sampled_word

        text_in.append(sampled_word)
    new_list=[]
    new_string=''
    meaning=[]
    nopunc = re.sub(r'[^\w\s]','', final) 
    final=[word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    for word in final:
        new_list.append('#' + word)
        for synset in wordnet.synsets(word):
          for lemma in synset.lemmas():
              new_list.append('#' + lemma.name())
              
    new_string =' \n'.join(set(new_list))
    print(final)
    print(new_string) 


    return render_template('after.html', data=new_string)

if __name__ == "__main__":
    app.run()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
++++++++++++++++++++++++++++++++++++++++++++++++++
vocabulary loaded
MODEL LOADED
RESNET MODEL LOADED
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://20f792f1f534.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Aug/2021 18:20:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2021 18:20:42] "GET /content/Finalmodel/image-captioning-keras-resnet/static/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Aug/2021 18:20:42] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Aug/2021 18:20:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2021 18:20:44] "GET /content/Finalmodel/image-captioning-keras-resnet/static/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Aug/2021 18:20:48] "GET /after HTTP/1.1" 400 -


IMAGE SAVED


0it [00:00, ?it/s]

Predict Features
GETING Captions



0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
127.0.0.1 - - [07/Aug/2021 18:20:56] "POST /after HTTP/1.1" 200 -


['group', 'greyhounds', 'busy', 'urban', 'street', 'like', 'something']
#the_likes_of 
#in_use 
#greyhound 
#busybodied 
#street 
#group 
#busy 
#greyhounds 
#something 
#corresponding 
#ilk 
#meddlesome 
#urban 
#like 
#same 
#engaged 
#care 
#radical 
#aggroup 
#grouping 
#similar 
#fussy 
#the_like 
#chemical_group 
#comparable 
#alike 
#officious 
#meddling 
#interfering 
#occupy 
#wish 
#mathematical_group


127.0.0.1 - - [07/Aug/2021 18:20:56] "GET /content/Finalmodel/image-captioning-keras-resnet/static/styles.css HTTP/1.1" 404 -
